In [ ]:
import zipfile

from PIL import Image
import pytesseract
import cv2 as cv
import numpy as np

# loading the face detection classifier
face_cascade = cv.CascadeClassifier('readonly/haarcascade_frontalface_default.xml')

# the rest is up to you!
#unzip the file
images = {}

zf = zipfile.ZipFile('readonly/images.zip')
for item in zf.infolist():
    images[item.filename] = Image.open(zf.open(item.filename))
imageinfo_list = list(images.values())
file_names = list(images.keys())
# print(imageinfo_list)
# print(file_names)
# Store the texts and faces of images
texts = []
faces = []
for key in images.keys():
    #print('convert to text')
    # convert to bilevel image and to texts
    bw_images = images[key].convert('1')
    texts.append(pytesseract.image_to_string(bw_images))
    
    #print('convert to opencv')
    # convert to opencv grayscale
    opencv_images = np.array(images[key])
    gray = cv.cvtColor(opencv_images, cv.COLOR_RGB2GRAY)
    
    #print('store faces')
    # detect and store faces
    face = face_cascade.detectMultiScale(gray, scaleFactor=1.3, minNeighbors=5, minSize=(40,40))
    faces.append(face)
    #print('Done for now')
    
def search(name):
    for i in range(len(texts)):
        if name in texts[i]:
            print('Results found in file {}'.format(file_names[i]))
            if len(faces[i]) != 0:
                
                # create a contact sheet
                face_height = 100
                rows_sheet = int(len(faces[i])/5)
                if len(faces[i])%5 != 0:
                    rows_sheet = rows_sheet + 1
                contact_sheet = Image.new('RGB', (500, 100*rows_sheet))
                image = imageinfo_list[i]
                image_x = 0
                image_y = 0
                for x, y, w, h in faces[i]:
                    face_img = image.crop((x, y, x+w, y+h))
                    face_img.thumbnail((face_width, face_height))
                    contact_sheet.paste(face_img, (image_x, image_y))

                    # next position
                    if image_x + face_width == contact_sheet.width:
                        image_x = 0
                        image_y = image_y + face_height
                    else:
                        image_x = image_x + face_width
                        
                display(contact_sheet)
            else:
                print('But there were no faces in that file!')
                
search('Christopher')
search('Mark')